In [3]:
# imports iniciales
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils
import calendar
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [4]:
events = utils.get_events()
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [5]:
# agrego columnas para segmentar el tiempo 
events.timestamp = pd.to_datetime(events['timestamp'],format='%Y-%m-%d %H:%M:%S')
events['month'] = events.timestamp.dt.month
events['day'] = events.timestamp.dt.day
events['hour'] = events.timestamp.dt.hour
events['weekday name'] = events.timestamp.dt.weekday_name
events[['person','timestamp','month','day','hour','weekday name']].head()

,person,timestamp,month,day,hour,weekday name
0,4886f805,2018-05-18 00:11:59,5,18,0,Friday
1,ad93850f,2018-05-18 00:11:27,5,18,0,Friday
2,0297fc1e,2018-05-18 00:11:16,5,18,0,Friday
3,2d681dd8,2018-05-18 00:11:14,5,18,0,Friday
4,cccea85e,2018-05-18 00:11:09,5,18,0,Friday


In [6]:
## 1 si es fin de semana 0 si es dia de semana
events['weekend'] = (events['weekday name'] == 'Saturday') | (events['weekday name'] == 'Sunday')
events['weekend'] = events['weekend'].astype(int)

In [7]:
## paso el mes a nombre
events['month'] = events['month'].apply(lambda x: calendar.month_abbr[x])

In [8]:
def periodioDia(hora):
    if (hora >= 0) & (hora < 7):
        return 'madrugada'
    elif (hora >= 7) & (hora < 12):
        return 'manana'
    elif (hora >= 12) & (hora < 15):
        return 'mediodia'
    elif (hora >= 15) & (hora < 20):
        return 'tarde'
    return 'noche'

events['periodo'] = events['hour'].apply(lambda x: periodioDia(x))

In [9]:
#one hot encoding
events = pd.concat([events, pd.get_dummies(events['weekday name'], prefix='weekday')],axis=1)
events = events.drop('weekday name', axis=1)
events = pd.concat([events, pd.get_dummies(events['periodo'], prefix='periodo')],axis=1)
events = events.drop('periodo', axis=1)
events = pd.concat([events, pd.get_dummies(events['month'], prefix='mes')],axis=1)
events = events.drop('month', axis=1)

events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,periodo_madrugada,periodo_manana,periodo_mediodia,periodo_noche,periodo_tarde,mes_Apr,mes_Feb,mes_Jan,mes_Mar,mes_May
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,1,0,0,0,0,0,0,0,0,1
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,1,0,0,0,0,0,0,0,0,1
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,1,0,0,0,0,0,0,0,0,1
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,1,0,0,0,0,0,0,0,0,1
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,1,0,0,0,0,0,0,0,0,1


In [10]:
events.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'day', 'hora', 'hour',
       'weekend', 'weekday_Friday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday', 'periodo_madrugada', 'periodo_manana',
       'periodo_mediodia', 'periodo_noche', 'periodo_tarde', 'mes_Apr',
       'mes_Feb', 'mes_Jan', 'mes_Mar', 'mes_May'],
      dtype='object')

In [11]:
events = events[['person',  'day', 'hora', 'hour', 'weekend', 'weekday_Friday',
       'weekday_Monday', 'weekday_Saturday', 'weekday_Sunday',
       'weekday_Thursday', 'weekday_Tuesday', 'weekday_Wednesday',
       'periodo_madrugada', 'periodo_manana', 'periodo_mediodia',
       'periodo_noche', 'periodo_tarde', 'mes_Apr', 'mes_Feb', 'mes_Jan',
       'mes_Mar', 'mes_May']]

In [12]:
features = events.groupby('person').sum().reset_index()
features.head()

,person,day,hora,hour,weekend,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,...,periodo_madrugada,periodo_manana,periodo_mediodia,periodo_noche,periodo_tarde,mes_Apr,mes_Feb,mes_Jan,mes_Mar,mes_May
0,0008ed71,102,86,86,0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0
1,00091926,7496,3139,3139,163,86.0,32.0,32.0,131.0,84.0,...,313.0,0.0,0.0,75.0,60.0,0.0,0.0,0.0,0.0,448.0
2,00091a7a,260,140,140,0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
3,000ba417,4586,2786,2786,59,0.0,0.0,59.0,0.0,147.0,...,0.0,57.0,68.0,0.0,81.0,0.0,0.0,0.0,0.0,206.0
4,000c79fe,493,0,0,0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0


In [13]:
features.shape

(38829, 22)

In [14]:
features.to_csv('features_dias.csv',index=False)